In [1]:
import requests
from bs4 import BeautifulSoup

import re

import pandas as pd

In [2]:
def get_html_content(url):
    url_website = requests.get(url)
    return BeautifulSoup(url_website.text, "html.parser")

In [3]:
eventDeptDict = {'Communications and Engagement':'D/Coms', 'Climate Initiatives':'D/CI', 'Housing and Human Services':'d/hhs', 'Open Space and Mountain Parks':'d/osmp', 'Parks and Recreation':'d/parksrec', 'Public Works':'d/pw',}

In [4]:
#df_deptEventHtml = pd.DataFrame(index=eventDeptDict.keys(),columns=["HTML code for the department"])
dict_deptEventHtml = {}

for dept in eventDeptDict.keys():
    dict_deptEventHtml[dept] = get_html_content("https://countmein.bouldercolorado.gov/"+eventDeptDict[dept])

In [5]:
dict_deptEventHtml.keys()

dict_keys(['Communications and Engagement', 'Climate Initiatives', 'Housing and Human Services', 'Open Space and Mountain Parks', 'Parks and Recreation', 'Public Works'])

In [6]:
dict_deptEventHtml["Parks and Recreation"]


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Count Me In: Department Landing Page</title>
<meta content="Count Me In is the volunteer management system and activity registration portal for City of Boulder - Volunteer Cooperative. Use the system to search for activities and log volunteer hours." name="description"/>
<meta content="Count Me In, volunteer management system, activity registration portal, City of Boulder - Volunteer Cooperative" name="keywords"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="text/css" http-equiv="Content-Style-Type">
<!-- Always force latest IE rendering engine (even in intranet) & Chrome Frame -->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="en" http-equiv="content-language">
<meta content="index,follow" name="robots">
<meta content="Count Me In" name="apple-mobile-web-app-title"/>
<link href="https:

In [7]:
event_html={}

for dept in dict_deptEventHtml.keys():
    event_html[dept] = [html for html in dict_deptEventHtml[dept].find_all('tr')]

In [25]:
eventDetails = pd.DataFrame(columns=["Event Name", "Date", "Time", "Event Description"])

for event_html_iter in event_html.values():
    for eventList in event_html_iter:
        event_name_element = eventList.find('a', attrs={"title": "Activity Details"})# or eventList.find('a', attrs={"href": re.compile("/PublicActivityVolunteerRegistration/")})
        date_time_element = eventList.find('h5', attrs={"class": "margin-top-5"})
        description_element = eventList.find('p')
        
        # Check if elements are found before calling get_text()
        event_name = event_name_element.get_text() if event_name_element else None
        date_time = date_time_element.get_text() if date_time_element else None
        date_time_regex = r"(\d{1,2}\s[A-Za-z]+\s\d{4})\s+\|\s+(\d{2}:\d{2}\s[AP]M)"
        date, time = re.findall(date_time_regex, date_time)[0]
        description = description_element.get_text() if description_element else None
        
        # Append data to DataFrame only if all elements are found
        if all((event_name, date_time, description)):
            eventRow = pd.DataFrame({"Event Name": [event_name], "Date": [date], "Time": [time], "Event Description": [description]})
            eventDetails = pd.concat([eventDetails, eventRow], ignore_index=True)

eventDetails

,Event Name,Date,Time,Event Description
0,\nLet's Doo It!: Dog Waste Clean Up\n,05 May 2024,09:00 AM,"\n Make your pup proud, and Let..."
1,\nWild Moms\n,05 May 2024,02:00 PM,\n While every mom is one-of-a-...
2,\nConnecting with Nature Through Art (Older Ad...,09 May 2024,09:00 AM,\n Join volunteer naturalists B...
3,\nRecovery and Wellness Series: Conscious Reco...,11 May 2024,09:00 AM,\n A program geared towards tho...
4,\nCANCELED: StoryTime: Hello Bees! (KIDS Ages ...,12 May 2024,10:00 AM,\n Let’s say hello to the bees ...
5,\nTrail Challenge Series: Flatirons Loop to En...,12 May 2024,09:00 AM,\n The City of Boulder has over...
6,\nFind Your Village: An Event with Hiking Baby...,14 May 2024,09:30 AM,\n Hitting the trails with litt...
7,\nWalk-n-Roll: An Accessible Spring Adventure ...,16 May 2024,10:00 AM,\n Enjoy a leisurely stroll thr...
8,\nOlder Adults West Boulder Neighborhood Walk\n,23 May 2024,09:00 AM,\n Come join Jane in an excitin...


In [9]:
eventDetails.replace('\n', '', regex=True, inplace=True)

In [10]:
eventDetails

,Event Name,Date,Time,Event Description
0,Let's Doo It!: Dog Waste Clean Up,05 May 2024,09:00 AM,"Make your pup proud, and Let’s..."
1,Wild Moms,05 May 2024,02:00 PM,While every mom is one-of-a-ki...
2,Volunteer,05 May 2024,09:00 AM,"Make your pup proud, and Let’s..."
3,Connecting with Nature Through Art (Older Adul...,09 May 2024,09:00 AM,Join volunteer naturalists Bev...
4,Recovery and Wellness Series: Conscious Reconn...,11 May 2024,09:00 AM,A program geared towards those...
5,CANCELED: StoryTime: Hello Bees! (KIDS Ages 3-6),12 May 2024,10:00 AM,Let’s say hello to the bees to...
6,Trail Challenge Series: Flatirons Loop to Ench...,12 May 2024,09:00 AM,The City of Boulder has over 4...
7,Find Your Village: An Event with Hiking Baby B...,14 May 2024,09:30 AM,Hitting the trails with little...
8,Volunteer,15 May 2024,09:00 AM,Volunteers will clean up litte...
9,Walk-n-Roll: An Accessible Spring Adventure at...,16 May 2024,10:00 AM,Enjoy a leisurely stroll throu...


In [11]:
import datetime
 
# using now() to get current time
current_time = datetime.datetime.now()

In [12]:
print(current_time)

2024-05-03 00:36:05.808864
